---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [38]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv',encoding = 'ISO-8859-1')
test = pd.read_csv('test.csv')
train.set_index('ticket_id')
train.columns

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [39]:
test.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [40]:
train.shape

(250306, 34)

In [41]:
test.shape

(61001, 27)

In [42]:
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [43]:
train[(train['compliance'] == 0.0) | (train['compliance'] == 1.0)].shape

(159880, 34)

In [48]:
address = pd.read_csv('addresses.csv')
address.head()
address.shape

(311307, 2)

In [49]:
latlons = pd.read_csv('latlons.csv')


In [50]:
address = pd.merge(address, latlons,how ='left')
address.shape

(311307, 4)

In [51]:
test = pd.merge(test,address,how='left')
train = pd.merge(train,address,how='left')
test.shape

(61001, 30)

In [52]:
train_df= train[(train['compliance'] == 0.0) | (train['compliance'] == 1.0)]
train_df.shape

(159880, 37)

In [53]:
train_df = train_df[~train_df['hearing_date'].isnull()]

In [54]:
train_df.shape

(159653, 37)

feature_to_be_splitted = ['agency_name', 'state', 'disposition']
train_df = pd.get_dummies(train_df,columns=feature_to_be_splitted)
test = pd.get_dummies(test,columns=feature_to_be_splitted)
train_df.shape

In [55]:
train_drop_list=['balance_due','payment_amount','payment_date','payment_status' ,'collection_status' ,'compliance_detail']
train_df.drop(train_drop_list,axis=1,inplace = True)


In [56]:
string_list = ['violator_name', 'zip_code', 'country', 'city',
            'inspector_name', 'violation_street_number', 'violation_street_name',
            'violation_zip_code', 'violation_description',
            'mailing_address_str_number', 'mailing_address_str_name',
            'non_us_str_code', 'agency_name', 'state', 'disposition',
            'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code','address']
train_df.drop(string_list,axis=1,inplace=True)
train_df.shape

(159653, 11)

In [57]:
test.drop(string_list,axis=1,inplace=True)

In [58]:
test.columns

Index(['ticket_id', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
       'discount_amount', 'clean_up_cost', 'judgment_amount', 'lat', 'lon'],
      dtype='object')

In [59]:
train_df.columns

Index(['ticket_id', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
       'discount_amount', 'clean_up_cost', 'judgment_amount', 'compliance',
       'lat', 'lon'],
      dtype='object')

In [60]:
train_df.lat.fillna(method = 'ffill',inplace=True)
train_df.lon.fillna(method = 'ffill',inplace=True)
test.lat.fillna(method = 'ffill',inplace=True)
test.lon.fillna(method = 'ffill',inplace=True)
test=test.set_index('ticket_id')
train_df= train_df.set_index('ticket_id')
test

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,lat,lon
ticket_id,,,,,,,,,
284932,200.0,20.0,10.0,20.0,0.0,0.0,250.0,42.407581,-82.986642
285362,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,42.426239,-83.238259
285361,100.0,20.0,10.0,10.0,0.0,0.0,140.0,42.426239,-83.238259
285338,200.0,20.0,10.0,20.0,0.0,0.0,250.0,42.309661,-83.122426
285346,100.0,20.0,10.0,10.0,0.0,0.0,140.0,42.308830,-83.121116
285345,200.0,20.0,10.0,20.0,0.0,0.0,250.0,42.308830,-83.121116
285347,50.0,20.0,10.0,5.0,0.0,0.0,85.0,42.308830,-83.121116
285342,200.0,20.0,10.0,0.0,0.0,0.0,230.0,42.313314,-83.108636
285530,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,42.261245,-83.160878


In [61]:
X_train = train_df[train_df.columns.drop('compliance')]
y_train = train_df['compliance']
X_test = test

In [62]:
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
model = MLPClassifier()

In [63]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = MLPClassifier(hidden_layer_sizes=[100,10],alpha=0.01,random_state=0,
                     solver='lbfgs',verbose=0)
model.fit(X_train_scaled,y_train)

y_proba = model.predict_proba(X_test_scaled)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
grid_values = ({'alpha':[0.01],'hidden_layer_sizes':[[100,10],[150,10]]})
grid_model = GridSearchCV(model,param_grid=grid_values,scoring='roc_auc')
grid_model.fit(X_train_scaled,y_train)

In [ ]:
y_proba = grid_model.predict_proba(X_test_scaled)[:,1]
test_df = pd.read_csv('test.csv',encoding = "ISO-8859-1")
test_df['compliance'] = y_proba
test_df.set_index('ticket_id',inplace=True)

In [36]:
def blight_model():
    
    return test_df.compliance
blight_model()

ticket_id
284932    0.052236
285362    0.013646
285361    0.055240
285338    0.056789
285346    0.066094
285345    0.056869
285347    0.071212
285342    0.293327
285530    0.017315
284989    0.034169
285344    0.070996
285343    0.016417
285340    0.016459
285341    0.071167
285349    0.066080
285348    0.056857
284991    0.034173
285532    0.037746
285406    0.031360
285001    0.035633
285006    0.016320
285405    0.013571
285337    0.029529
285496    0.065524
285497    0.052263
285378    0.013579
285589    0.029551
285585    0.047960
285501    0.060961
285581    0.013527
            ...   
376367    0.021394
376366    0.046430
376362    0.045269
376363    0.048859
376365    0.021394
376364    0.046430
376228    0.051546
376265    0.045498
376286    0.345691
376320    0.050006
376314    0.051201
376327    0.380166
376385    0.365788
376435    0.551080
376370    0.381029
376434    0.062764
376459    0.063734
376478    0.000005
376473    0.050612
376484    0.047273
376482    0.031213
37